In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning import Trainer
import wandb
import math
sys.path.append('../..')

from LightningModules.GNNEmbeddings.Models.models import MLPEmbedding, InteractionGNN, HierarchicalGNN, SparseHierarchicalGNN

device = "cuda" if torch.cuda.is_available() else "cpu"
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
def kaiming_init(model):
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            param.data.fill_(0)
        elif name.startswith("layers.0"):  # The first layer does not have ReLU applied on its input
            param.data.normal_(0, 1 / math.sqrt(param.shape[1]))
        else:
            param.data.normal_(0, math.sqrt(2) / math.sqrt(param.shape[1]))

In [4]:
def load_from_pretrained(model, path):
    
    checkpoint = torch.load(path)
    state_dict = checkpoint["state_dict"]
    names = [i for i in state_dict]
    for i in names:
        if "encoder" in i:
            del state_dict[i]
    model.load_state_dict(state_dict, strict=False)
    del state_dict
    
    return model

## Construct PyLightning model

In [5]:
with open("embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
if hparams["use_toy"]:
    hparams["regime"] = []
    hparams["spatial_channels"] = 2
if hparams["model"] == "interaction" or hparams["model"] == 1:
    model = InteractionGNN(hparams)
elif hparams["model"] == "hierarchical" or hparams["model"] == 2:
    model = HierarchicalGNN(hparams)
elif hparams["model"] == "mlp" or hparams["model"] == 3:
    model = MLPEmbedding(hparams)
elif hparams["model"] == "sparsehierarchical" or hparams["model"] == 4:
    model = SparseHierarchicalGNN(hparams)
    
model_path = "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_barrel_embedding/ITk_barrel_embedding/yhex7nae/checkpoints/last.ckpt"
if hparams["use_pretrain"]:
    model = load_from_pretrained(model, model_path)

## Metric Learning

In [7]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='eff',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
# kaiming_init(model)

logger = WandbLogger(project="ITk_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/")
trainer.fit(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: liuryan30 (exatrkx). Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                    | Type                           | Params
---------------------------------------------------------------------------
0 | node_encoder            | Sequential                     | 100 K 
1 | edge_encoder            | Sequential                     | 101 K 
2 | supergraph_construction | SparseHierarchicalGNNAttention | 269 K 
3 | ignn_blocks             | ModuleList                     | 1.5 M 
4 | hgnn_blocks             | ModuleList                     | 4.8 M 
5 | output_layer            | Sequential                     | 103 K 
6 | softmax                 | Softmax                        | 0     
---------------------------------------------------------------------------
6.8 M     Trainable params
0         Non-trainable params
6.8 M     Total params
27.190    Total est

Sanity Checking: 0it [00:00, ?it/s]

/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


time out!Label prop iterations: 13

Label prop iterations: 6
Label prop iterations: 7
Label prop iterations: 6
Label prop iterations: 3
Label prop iterations: 2
Iterations: 6
14409,222,233,28,377,1491
Label prop iterations: 4
Label prop iterations: 4
Label prop iterations: 2
Iterations: 3
595,117,117,13,172,456
Label prop iterations: 2
Iterations: 1
571,59,58,6,60,119


...

Label prop iterations: 11
Label prop iterations: 8
Label prop iterations: 4
Label prop iterations: 4
Label prop iterations: 2
Label prop iterations: 2
Iterations: 6
902,225,200,28,514,1441
Label prop iterations: 7
Label prop iterations: 5
Label prop iterations: 3
Iterations: 3
715,109,145,13,237,663
Label prop iterations: 2
Iterations: 1
608,58,53,6,70,120
Label prop iterations: 14
Label prop iterations: 8
Label prop iterations: 7
Label prop iterations: 6
Label prop iterations: 6
Iterations: 5
869,183,192,27,403,1669
Label prop iterations: 4
Label prop iterations: 3
Label prop iterations: 2
Iterations: 3
687,113,114,12,257,508


/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7343. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Label prop iterations: 11
Label prop iterations: 5
Label prop iterations: 3
Label prop iterations: 3
Label prop iterations: 2
Iterations: 5
747,165,169,22,339,1015
Label prop iterations: 7
Label prop iterations: 7
Label prop iterations: 3
Iterations: 3
625,104,125,11,190,647
Label prop iterations: 2
Iterations: 1
552,52,55,5,54,110
Label prop iterations: 13
Label prop iterations: 7
Label prop iterations: 7
Label prop iterations: 6
Label prop iterations: 6
Label prop iterations: 2
Iterations: 6
838,205,199,32,477,1698
Label prop iterations: 6
Label prop iterations: 3
Label prop iterations: 2
Iterations: 3
632,109,108,12,210,551


/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7017. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Label prop iterations: 12
Label prop iterations: 5
Label prop iterations: 5
Label prop iterations: 2
Label prop iterations: 2
Iterations: 5
823,167,157,20,370,1112
Label prop iterations: 7
Label prop iterations: 5
Label prop iterations: 3
Label prop iterations: 2
Iterations: 4
639,126,155,13,284,729
Label prop iterations: 2
Iterations: 1
543,51,48,5,73,111
Label prop iterations: 13
Label prop iterations: 9
Label prop iterations: 7
Label prop iterations: 5
Label prop iterations: 4
Label prop iterations: 2
Iterations: 6
951,217,219,29,590,1891
Label prop iterations: 4
Label prop iterations: 3
Label prop iterations: 2
Iterations: 3
696,116,117,12,229,487
Label prop iterations: 14
Label prop iterations: 7
Label prop iterations: 6
Label prop iterations: 9
Label prop iterations: 5
Label prop iterations: 2
Iterations: 6
612,175,142,22,406,1264
Label prop iterations: 4
Label prop iterations: 3
Label prop iterations: 2
Iterations: 3
472,97,77,13,198,356
Label prop iterations: 15
Label prop iter

## Sweep

In [5]:
run_name = input()

 test


In [6]:
with open("embedding_sweep.yaml") as f:
        sweep_hparams = yaml.load(f, Loader=yaml.FullLoader)
with open("embedding_default.yaml") as f:
        default_hparams = yaml.load(f, Loader=yaml.FullLoader)

In [7]:
sweep_configuration = {
    "name": run_name,
    "project": "ITk_barrel_embedding",
    "metric": {"name": "track_eff", "goal": "maximize"},
    "method": "grid",
    "parameters": sweep_hparams
}

In [8]:
def training():
    wandb.init()
    hparams = {**default_hparams, **wandb.config}
    if hparams["use_toy"]:
        hparams["regime"] = []
        hparams["spatial_channels"] = 2
        
    if hparams["model"] == "interaction" or hparams["model"] == 1:
        model = InteractionGNN(hparams)
        model_path = "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_barrel_embedding/ITk_barrel_embedding/zi7bos0a/checkpoints/last.ckpt"
    elif hparams["model"] == "hierarchical" or hparams["model"] == 2:
        model = HierarchicalGNN(hparams)
        model_path = "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_barrel_embedding/ITk_barrel_embedding/neoc2rn0/checkpoints/last.ckpt"
    elif hparams["model"] == "mlp" or hparams["model"] == 3:
        model = MLPEmbedding(hparams)
        model_path = "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_barrel_embedding/ITk_barrel_embedding/f99yr0vb/checkpoints/last.ckpt"
    elif hparams["model"] == "sparsehierarchical" or hparams["model"] == 4:
        model = SparseHierarchicalGNN(hparams)
        model_path = "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_barrel_embedding/ITk_barrel_embedding/yhex7nae/checkpoints/last.ckpt"
    
    if hparams["use_pretrain"]:
        model = load_from_pretrained(model, model_path)
    
    checkpoint_callback = ModelCheckpoint(
        monitor='track_eff',
        mode="max",
        save_top_k=2,
        save_last=True)

    logger = WandbLogger()
    trainer = Trainer(gpus=1, max_epochs=default_hparams["max_epochs"], log_every_n_steps = 50, logger=logger, callbacks=[checkpoint_callback], default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_barrel_embedding/")
    trainer.fit(model)

In [ ]:
sweep_id = wandb.sweep(sweep_configuration, project = "ITk_barrel_embedding")

# run the sweep
wandb.agent(sweep_id, function=training)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 3cpru6eg
Sweep URL: https://wandb.ai/exatrkx/ITk_barrel_embedding/sweeps/3cpru6eg


wandb: Agent Starting Run: vk6hptlw with config:
wandb: 	model: sparsehierarchical
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: liuryan30 (exatrkx). Use `wandb login --relogin` to force relogin


/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                    | Type       | Params
-------------------------------------------------------
0 | node_encoder            | Sequential | 100 K 
1 | edge_encoder            | Sequential | 101 K 
2 | supergraph_construction | ModuleList | 535 K 
3 | ignn_blocks             | ModuleList | 1.5 M 
4 | hgnn_blocks             | ModuleList | 4.8 M 
5 | output_layer            | Sequential | 101 K 
6 | softmax                 | Softmax    | 0     
-------------------------

Sanity Checking: 0it [00:00, ?it/s]

...

Label prop iterations: 14
Label prop iterations: 8
Label prop iterations: 7
Label prop iterations: 3
Label prop iterations: 6
Iterations: 5
528,147,112,21,234,1113
Label prop iterations: 4
Label prop iterations: 3
Iterations: 2
371,68,51,9,91,221


Training: 0it [00:00, ?it/s]

Label prop iterations: 14
Label prop iterations: 7
Label prop iterations: 7
Label prop iterations: 6
Label prop iterations: 2
Iterations: 5
820,183,160,23,313,1383
Label prop iterations: 3
Label prop iterations: 5
Iterations: 2
581,87,74,9,115,351


/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:229: UserWarning: You called `self.log('clusters', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Label prop iterations: 13
Label prop iterations: 8
Label prop iterations: 6
Label prop iterations: 5
Label prop iterations: 4
Iterations: 5
799,199,162,34,327,1573
Label prop iterations: 2
Label prop iterations: 2
Iterations: 2
551,91,74,11,121,248
Label prop iterations: 19
Label prop iterations: 9
Label prop iterations: 8
Label prop iterations: 5
Label prop iterations: 3
Iterations: 5
696,174,147,21,291,1605
Label prop iterations: 5
Label prop iterations: 4
Iterations: 2
530,82,73,9,113,377
Label prop iterations: 12
Label prop iterations: 10
Label prop iterations: 7
Label prop iterations: 5
Label prop iterations: 3
Label prop iterations: 2
Iterations: 6
543,172,130,24,301,1130
Label prop iterations: 3
Label prop iterations: 4
Iterations: 2
407,70,49,9,97,219
Label prop iterations: 17
Label prop iterations: 8
Label prop iterations: 8
Label prop iterations: 4
Label prop iterations: 3
Iterations: 5
604,172,120,24,293,1214
Label prop iterations: 4
Label prop iterations: 3
Iterations: 2
40

## Initialize from trained model

In [ ]:
import wandb
wandb.finish()

with open("dual_embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)
    
hparams["use_dual_encoder"] = True

model = VanillaDualEmbedding(hparams)

checkpoint = torch.load("/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/ITk_dual_embedding/3ijb4qnw/checkpoints/last.ckpt")
state_dict = checkpoint["state_dict"]
names = [i for i in state_dict]
for i in names:
    state = state_dict[i]
    i = i.replace("input_layer1", "input_layer2")
    i = i.replace("layers1", "layers2")
    i = i.replace("output_layer1", "output_layer2")
    state_dict[i] = state

model.load_state_dict(state_dict)
del state_dict

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='pur',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
logger = WandbLogger(project="ITk_dual_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/")
trainer.fit(model)